In [1]:
import json
from sklearn.externals import joblib
import random
import numpy as np

In [2]:
events_num, weibos_num, event_weibos_num_list, global_sampling_factor, event_sampling_factor_list = joblib.load(
        'file/pkl/event_sampling_factor.pkl')
sampling_num_of_event, certify_num_of_event = joblib.load('file/pkl/certify_num_of_every_event.pkl')
event_features_list = joblib.load('file/pkl/event_features.pkl')

In [4]:
events_num, weibos_num

(5979, 38180)

In [7]:
sum(sampling_num_of_event)

6753

In [9]:
sum([sum(x) for x in certify_num_of_event])

6753

# 计算参数

In [53]:
# （5942）4439:1284:219 = 20.3 : 5.9 : 1
                
# Rumor Features
rumor_weibo_sum = 7880
rumor_certify = 5942
rumor_0 = 4439
rumor_1 = 1284
rumor_2 = 219

In [317]:
def get_event_certify_num():
    events_num, weibos_num, event_weibos_num_list, global_sampling_factor, event_sampling_factor_list = joblib.load(
        'file/pkl/event_sampling_factor.pkl')
    # 计算在每个事件中抽取的微博数
    sampling_num_of_event = [weibos_num * global_sampling_factor * factor for factor in event_sampling_factor_list]
    sampling_num_of_event = [int(x + 0.5) for x in sampling_num_of_event]
    sampling_num_of_event = [1 if x == 0 else x for x in sampling_num_of_event]

    # 计算不同的userCertify字段应该分别抽取多少微博
    certify_num_of_event = []
    for x in sampling_num_of_event:
        certify_num = [rumor_0 / rumor_certify * x, rumor_1 / rumor_certify * x, rumor_2 / rumor_certify * x]
        
        # 有问题的四舍五入，应舍弃
#         certify_num = [int(x + 0.4) for x in certify_num]

        certify_num_of_event.append(certify_num)

        # 按照一定的概率，减少certify0与certify1
#         updated_certify_num = []
#         for i in range(3):
#             num = certify_num[i]
#             rand = random.random()
#             if rand >= 0.7 and i < 2:
#                 num = num - 1 if num > 0 else num
#             rand = random.random()
#             if rand >= 0.95 and i == 2:
#                 num += 1
#             updated_certify_num.append(num)
            
#         num_0 = int(certify_num[0] + 0.5)
#         num_1 = int(certify_num[1] + 0.5)
#         num_2 = int(certify_num[2] + 1)
#         certify_num_of_event.append([num_0, num_1, num_2])

    # 更新在每个事件中抽取的微博数
    sampling_num_of_event = [sum(x) for x in certify_num_of_event]

    joblib.dump((sampling_num_of_event, certify_num_of_event), 'file/pkl/certify_num_of_every_event.pkl')

    
get_event_certify_num()

In [318]:
sampling_num_of_event, certify_num_of_event = joblib.load('file/pkl/certify_num_of_every_event.pkl')

In [319]:
sum(sampling_num_of_event)

9839.0

In [320]:
certify_num_of_event[:5]

[[0.7470548636822618, 0.21608885897004376, 0.03685627734769438],
 [3.7352743184113093, 1.0804442948502189, 0.1842813867384719],
 [0.7470548636822618, 0.21608885897004376, 0.03685627734769438],
 [0.7470548636822618, 0.21608885897004376, 0.03685627734769438],
 [2.2411645910467852, 0.6482665769101312, 0.11056883204308314]]

In [321]:
c0 = sum([x[0] for x in certify_num_of_event])
c1 = sum([x[1] for x in certify_num_of_event])
c2 = sum([x[2] for x in certify_num_of_event])

print('({}) {}:{}:{} = {:.1f} : {:.1f} : 1'.format(c0+c1+c2, c0, c1,c2, c0/c2, c1/c2))

(9839.000000000042) 7350.2728037697125:2126.098283406401:362.6289128239282 = 20.3 : 5.9 : 1


# 查看result

In [42]:
result_certify_num_of_event, result_index_of_event = joblib.load('file/pkl/result.pkl')

In [44]:
result_certify_num_of_event[:5]

[[2, 0, 0], [3, 1, 0], [0, 0, 0], [0, 0, 0], [2, 1, 0]]